In [46]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> Download
Command 'Download' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> tokenizers/punkt/english.pickle
    Error loading tokenizers/punkt/english.pickle: Package
        'tokenizers/punkt/english.pickle' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> l

Packages:
  [ ] abc..

Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
        C:\Users\Ameera\AppData\Roaming\nltk_data...
      Unzipping tokenizers\punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
# Construct a dictionary mapping from word -> [# syllables], [# syllables when at the end of the word]
syl_data = np.loadtxt('./data/Syllable_dictionary.txt', dtype='str', delimiter='\n')
syl_dict = {}
for line in syl_data:
    syl_list = line.split()
    word = syl_list[0]
    syl_dict[word] = [[],[]]
    for i in range(1, len(syl_list)):
        if syl_list[i].startswith('E'):
            syl_dict[word][1].append(int(syl_list[i].lstrip('E')))
        else:
            syl_dict[word][0].append(int(syl_list[i]))

In [43]:
sonnet_data = np.loadtxt('./data/shakespeare.txt', dtype='str', delimiter='\n')

In [134]:
# Construct a sequence of tokens from sonnets.
sonnets = []
for line in sonnet_data:
    if line.lstrip().isdigit():
        sonnets.append([])
    else:
        tokens = word_tokenize(line)
        for i, token in enumerate(tokens):
            if (not token.lower() in syl_dict) and (token.lstrip("'").lower() in syl_dict):
                sonnets[-1].append(token.lstrip("'").lower())
            elif (not token.lower() in syl_dict) and (token.lower() + "'" in syl_dict):
                sonnets[-1].append(token.lower() + "'")
            elif i != len(tokens) - 1 and (tokens[i+1] == "'s" or tokens[i+1] == "'ll"):
                sonnets[-1].append(token.lower() + tokens[i+1])
            elif i != len(tokens) - 1 and tokens[i+1] == "is" and tokens[i].lower() == "'t":
                sonnets[-1].append("'tis")
            elif token in ["'s", "'", "(", ")", "'ll"]:
                continue
            elif i != 0 and tokens[i-1].lower() == "'t" and tokens[i].lower() == "is":
                continue
            else:
                sonnets[-1].append(token.lower())

In [102]:
syl_dict['grossly'] = [[2], []]
syl_dict['fickle'] = [[2], []]
syl_dict['waning'] = [[2], []]
syl_dict["show'st"] = [[1], []]
syl_dict['over'] = [[2], []]
syl_dict['wrack'] = [[1], []]
syl_dict['withering'] = [[2], []]
syl_dict['goest'] = [[1], []]
syl_dict['onwards'] = [[2], []]
syl_dict['minion'] = [[2], []]
syl_dict['detain'] = [[2], []]
syl_dict['delayed'] = [[2], []]
syl_dict['answered'] = [[2], []]
syl_dict['quietus'] = [[3], []]

In [180]:
for punct in ['.',',',':', '?', '!', ';']:
    syl_dict[punct] = [[0],[]]

In [135]:
for j, sonnet in enumerate(sonnets):
    for i, word in enumerate(sonnet):
        if not word in syl_dict and not word in ['.',',',':', '?', '!', ';']:
            print(str(j) + ' ' + str(i) + ' ' + word)

In [153]:
num_tokens = 0
token_dict = {}
num_dict = {}
for j, sonnet in enumerate(sonnets):
    for token in sonnet:
        if not token in token_dict:
            token_dict[token] = num_tokens
            num_dict[num_tokens] = token
            num_tokens += 1 

In [137]:
from hmmlearn.hmm import MultinomialHMM

In [251]:
hmm = MultinomialHMM(n_components=100, startprob_prior=1.0, transmat_prior=1.0, algorithm='viterbi', n_iter=10)

In [252]:
flat_sonnets = np.array([token_dict[token] for sonnet in sonnets for token in sonnet]).reshape(-1,1)
sonnet_lengths = np.array([len(sonnet) for sonnet in sonnets])
hmm.fit(flat_sonnets, sonnet_lengths)

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=100,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x00000226A74A1EE8>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=False)

In [277]:
# Return a token with the first alphanumeric character capitalized.
def capitalize(token):
    for i, c in enumerate(token):
        if c.isalnum():
            break
    return token[0:i] + c.upper() + token[i+1:(len(token))]           

# Turns the sonnet lines into a sonnet.
def process_lines(sonnet_lines, punct):
    processed_lines = [[] for i in range(len(sonnet_lines))]
    for i, line in enumerate(sonnet_lines):
        first_word_idx = i-1 if line[0] in punct else i
        if first_word_idx != -1:
            processed_lines[first_word_idx].append(line[0])
        for j in range(1, len(line)):
            processed_lines[i].append(line[j])
    
    for line in processed_lines:
        line[0] = capitalize(line[0])
        for i, token in enumerate(line):
            if token in ['.', '!', '?'] and i != len(line) - 1:
                line[i+1] = capitalize(line[i+1])
            elif token in ['i', "i'll"]:
                line[i] = capitalize(line[i])
    
    if processed_lines[-1][-1] != '.':
        processed_lines[-1].append('.')
    
    joined_lines = [' '.join(tokens) for tokens in processed_lines]
    return '\n'.join(joined_lines)

# Generates a sonnet given a trained hmm and a dictionary mapping numbers to words and words to syllables
def generate_sonnet(hmm, syl_dict, num_dict):
    '''
    Inputs:
        hmm: trained HMM
        num_dict: maps token number to token
        syl_dict: maps token to possible number of syllables
    '''
    punctuation = ['.',',',':', '?', '!', ';']
    emissions, states = hmm.sample(200)
    sonnet_lines = [[] for i in range(14)]
    state = 0
    prev_token_is_punct = False
    for i in range(14):
        line = sonnet_lines[i]
        # List of possible number of syllables in the line so far
        possible_syls = [0]
        while True:
            token_list = hmm._generate_sample_from_state(states[state])
            token_num = token_list[0]
            token = num_dict[token_num]
            token_syls = syl_dict[token]
            next_possible_syls = []
            for num_syls in possible_syls:
                for syls in token_syls[0] + token_syls[1]:
                    next_possible_syls.append(syls + num_syls)

            # Generate another token and try again with the same state. 
            if min(next_possible_syls) > 10:
                continue
            
            cur_token_is_punct = token in punctuation
            if prev_token_is_punct and cur_token_is_punct:
                continue
            
            prev_token_is_punct = cur_token_is_punct
                  
            # Line is complete, go to next line.
            if 10 in next_possible_syls:
                line.append(token)
                break           
            
            # In this case, the line is not complete. Add the token to the sonnet,
            # update possible_syls, and increment state.
            line.append(token)
            state += 1
            possible_syls = [a + b for a in possible_syls for b in token_syls[0]]
    return process_lines(sonnet_lines, punctuation)

In [279]:
print(generate_sonnet(hmm,syl_dict, num_dict))

Pleasure , her and to I ill bay breath it
Fair a nature be thee , sum , the who of
His love by not it are then , skill of and
Present face , who where thy see usurer
And away with counted with lend thee once
Tell replete by enjoyer and love of
He to shalt ? To I remedy well when
Belongs once bail ransom but to and it ,
Tender an still he to the story thoughts
When smell of tongues come linger body better
Love where , my looks being self self-doing :
From with but , with not my on increase did
In am record no the of the , I the
Beauty's husbandry true heaven your cools .


In [221]:
capitalize("i")

'I'

In [171]:
print([c + d for c in a for d in b])

[4, 5, 5, 6]


In [211]:
print(this[-2:])

is
